In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Create a folder in your Google Drive
project_path = "/content/drive/MyDrive/Measles_Chatbot_Project"
os.makedirs(project_path, exist_ok=True)

print(f"📂 Project folder created at: {project_path}")

📂 Project folder created at: /content/drive/MyDrive/Measles_Chatbot_Project


In [ ]:
# Install Hugging Face libraries
!pip install -q -U accelerate peft transformers trl
!pip install -q datasets
!pip install -U bitsandbytes


In [ ]:
import json
from datasets import Dataset

# Example of IDSR-aligned data (You must replace this with your actual IDSR data)
data = [

  {
    "instruction": "What is the standard case definition for a suspected measles case?",
    "context": "IDSR 3rd Edition - Standard Case Definitions (General Knowledge aligned with Section 11 header).",
    "response": "A suspected measles case is defined as any person with fever and maculopapular (non-vesicular) generalized rash and cough, coryza (runny nose), or conjunctivitis (red eyes), OR any person in whom a clinician suspects measles."
  },
  {
    "instruction": "What are the immediate reporting requirements for a suspected measles case?",
    "context": "IDSR 3rd Edition - Section 2: Reporting.",
    "response": "Measles is an epidemic-prone disease requiring immediate reporting. Any suspected case must be reported to the next level within 24 hours using the Case-based Reporting Form. It should also be notified via the fastest means possible (e.g., phone or SMS)."
  },
  {
    "instruction": "What is the specific purpose of the Euroimmun Anti-Measles ELISA test?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 1: Purpose.",
    "response": "The Euroimmun Anti-Measles ELISA test is designed for the semi-quantitative in vitro determination of human IgM class antibodies against the Measles virus nucleoprotein in serum or plasma. It is used to confirm a recent measles infection."
  },
  {
    "instruction": "What sample types are accepted for the Measles IgM ELISA test?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 4: Specimen.",
    "response": "The test accepts human serum, or plasma collected using EDTA, heparin, or citrate. Samples should not be lipemic, hemolytic, or icteric."
  },
  {
    "instruction": "How should serum samples be stored if testing is not performed immediately?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 4: Specimen.",
    "response": "If testing is performed within 1 week, samples can be stored at 2°C to 8°C. For longer storage, samples must be frozen at -20°C or colder. Frost-free freezers should be avoided to prevent freeze-thaw cycles which degrade antibodies."
  },
  {
    "instruction": "What are the critical safety precautions when handling reagents and samples for the Measles ELISA?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 5: Safety.",
    "response": "Staff must wear gloves and PPE. All blood and serum must be handled as infectious. The first three steps of the procedure (sample preparation) must be performed in a biosafety cabinet to prevent aerosol exposure. Surfaces should be disinfected with 1% Virkon or 70% Alcohol."
  },
  {
    "instruction": "How do you prepare the wash buffer for the Measles ELISA test?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Dilute the 10X washing buffer concentrate with distilled water at a 1:10 ratio. For example, to prepare enough for one microplate strip, mix 5 ml of 10X wash buffer concentrate with 45 ml of distilled water."
  },
  {
    "instruction": "What is the exact dilution ratio for patient samples in the Measles ELISA test?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Patient samples must be diluted 1:101. This is achieved by mixing 10 µl of the patient sample with 1000 µl (1.0 ml) of the sample buffer containing RF-Absorbent."
  },
  {
    "instruction": "Do the kit controls (positive, negative, calibrator) require dilution?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "No, the kit controls are ready for use and must NOT be diluted."
  },
  {
    "instruction": "What is the incubation protocol for the diluted test samples before adding them to the plate?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "After mixing the sample with the buffer, the diluted test samples must be incubated for 10 minutes at room temperature (18°C–25°C) before being dispensed into the microtiter wells."
  },
  {
    "instruction": "Describe the sample incubation step on the microplate.",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Add 100 µl of diluted samples and undiluted controls to the appropriate wells. Incubate the plate for 30 minutes at room temperature (18°C–25°C)."
  },
  {
    "instruction": "What is the washing procedure after the first incubation?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Wash all wells with 300 µl of diluted washing solution (450 µl if using an automated washer). Leave the buffer in wells for 30–60 seconds, then aspirate or shake out. Repeat this process for a total of 3 washes. Blot the plate on a paper towel to remove remaining liquid."
  },
  {
    "instruction": "How long is the conjugate incubation step?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Add 100 µl of enzyme conjugate to each well and incubate for 30 minutes at room temperature."
  },
  {
    "instruction": "What are the conditions for the substrate incubation?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Add 100 µl of chromogen/substrate solution to each well and incubate for 15 minutes at room temperature. It is critical to protect the plate from direct light during this step."
  },
  {
    "instruction": "How should the results be read after adding the stop solution?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 7: Procedure.",
    "response": "Read the Optical Density (OD) at 450 nm within 30 minutes of adding the stop solution. A reference wavelength between 620 nm and 650 nm (e.g., 630 nm) should be used."
  },
  {
    "instruction": "How are the results of the Measles ELISA interpreted?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 10: Reference Range.",
    "response": "Results are interpreted using a ratio based on the calibrator:\n- Ratio < 0.8: Negative\n- Ratio ≥ 0.8 to < 1.1: Equivocal (Borderline)\n- Ratio ≥ 1.1: Positive"
  },
  {
    "instruction": "What should be done if a sample yields an equivocal result?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 10: Reference Range (Standard interpretation).",
    "response": "If a result is equivocal (ratio between 0.8 and 1.1), the test should be repeated using a fresh sample collected 7 to 10 days later."
  },
  {
    "instruction": "What is the procedure if a suspected measles sample tests negative for measles IgM?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 12: Notes.",
    "response": "If a suspected measles sample tests negative for anti-measles virus IgM antibodies, it must then be tested for anti-rubella virus IgM antibodies."
  },
  {
    "instruction": "How should waste generated from the ELISA test be disposed of?",
    "context": "VIR-TECH-011-v2 Detection of Anti-Measles IgM Antibodies, Section 13: Disposal.",
    "response": "All waste must be placed securely in a biohazard bag and autoclaved at 121°C for 15 minutes before being sent for incineration."
  },
  {
    "instruction": "What does a 'confirmed' measles outbreak look like?",
    "context": "IDSR 3rd Edition - Section 11 (General Thresholds).",
    "response": "A confirmed measles outbreak is typically defined as 3 or more IgM positive measles cases detected in a health facility or district within a single month."
  },
  {
    "instruction": "Who should be involved in Community-Based Surveillance (CBS) for measles?",
    "context": "IDSR 3rd Edition - Section 1: Community Surveillance.",
    "response": "Community health workers, traditional healers, birth attendants, community leaders, and volunteers should be involved. They should be trained to recognize 'fever with rash' and report suspected cases immediately to the nearest health facility."
  },
  {
    "instruction": "What is the Alert Threshold for measles?",
    "context": "IDSR 3rd Edition - Section 11 (General Thresholds).",
    "response": "The alert threshold for measles is a single (1) suspected case. Even one case requires immediate investigation to confirm or rule out an outbreak."
  },
  {
    "instruction": "When should Vitamin A be administered to a measles patient?",
    "context": "IDSR 3rd Edition - Measles Treatment Guidelines.",
    "response": "Vitamin A should be administered immediately upon diagnosis of measles. A second dose is given the next day, and a third dose is given 2–4 weeks later if the child has clinical signs of Vitamin A deficiency."
  },
  {
    "instruction": "What is the recommended Vitamin A dose for an infant under 6 months?",
    "context": "IDSR 3rd Edition - Measles Treatment Guidelines.",
    "response": "For infants less than 6 months of age, the recommended Vitamin A dose is 50,000 IU."
  },
  {
    "instruction": "What is the recommended Vitamin A dose for children aged 12 months and older?",
    "context": "IDSR 3rd Edition - Measles Treatment Guidelines.",
    "response": "For children aged 12 months and older, the recommended Vitamin A dose is 200,000 IU."
  }

]

# Convert the list of dictionaries to a Hugging Face Dataset
dataset = Dataset.from_list(data)

# Define the formatting function to turn the data into a training prompt
def format_instruction(sample):
    # This template is used by the SFTTrainer to convert the data columns into a single string for training
    return f"""### Instruction:
{sample['instruction']}

### Context:
{sample['context']}

### Response:
{sample['response']}"""

# Apply the formatting function
dataset = dataset.map(lambda x: {'text': format_instruction(x)})

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

# Ensure bitsandbytes is up-to-date right before use
!pip install -U bitsandbytes

# 1. Quantization Configuration (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # NormalFloat 4-bit, which is optimal for LLM weights
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float32 # Compute in float32 to avoid BFloat16 issues
)

# 2. Load the base LLM (Mistral 7B)
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically handles model placement on the GPU
    dtype=torch.float32 # Explicitly set model's default data type to float32
)
model.to('cuda') # Explicitly move model to CUDA
# Note: You may need to log in to Hugging Face to access the model
# from huggingface_hub import notebook_login; notebook_login()

# 3. LoRA Configuration
peft_config = LoraConfig(
    r=8, # LoRA rank: reduced from 16 to 8 for memory optimization
    lora_alpha=32, # Scaling factor for LoRA updates
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Target common linear layers in the model
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# 1. Training Arguments
training_args = TrainingArguments(
    output_dir=f"{project_path}/adapter_weights", # Directory to save the checkpoint
    num_train_epochs=3, # Train for 3 epochs (Adjust based on dataset size)
    per_device_train_batch_size=2, # Adjust based on GPU memory
    gradient_accumulation_steps=2, # Increase this to simulate a larger batch size if memory is an issue
    optim="paged_adamw_8bit", # Memory-efficient AdamW
    learning_rate=2e-4, # Standard learning rate for LoRA
    logging_steps=10,
    save_strategy="epoch",
    fp16=False, # Use 16-bit floating point precision
    bf16=False, # Explicitly disable BFloat16 mixed precision
    gradient_checkpointing=False, # Disable gradient checkpointing to avoid potential device issues
)

# 2. Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)

# 3. Start Training
trainer.train()

# 4. Save the Adapter Weights
trainer.model.save_pretrained("mistral-7b-measles-qlora-adapter")

Adding EOS to train dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
10,1.239200
20,0.377800


wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run
wandb: WARNING URL not available in offline run


In [ ]:
from peft import PeftModel

# Load the base model (quantized)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load the LoRA adapter on top of the base model
ft_model = PeftModel.from_pretrained(base_model, "mistral-7b-measles-qlora-adapter")
ft_model = ft_model.merge_and_unload() # Optional: Merge the adapter weights back into the base model weights

# Example Inference
prompt = "### Instruction: What is the probable case definition for measles? ### Context: [Your retrieved IDSR text snippets] ### Response:"

inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
outputs = ft_model.generate(**inputs, max_new_tokens=256)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction: What is the probable case definition for measles? ### Context: [Your retrieved IDSR text snippets] ### Response: The probable case definition for measles, according to the Integrated Disease Surveillance and Response (IDSR) guidelines, is as follows:

1. **Fever (≥38.0°C) or history of fever**: Measles is a viral disease characterized by fever and rash. Fever is a common symptom and can be present or reported retrospectively.

2. **Generalized maculopapular rash**: The rash starts on the face and progresses downwards, usually appearing 3-5 days after the onset of fever. It is a characteristic feature of measles and is often described as a 'maculopapular' rash, meaning it consists of small red spots (macules) that merge into larger red areas (papules).

3. **Cough**: Coughing is a common symptom in measles, and it can be severe enough to cause difficulty in breathing in some cases.

4. **Arthralgia (joint pain) or conjunctivitis (inflammation of the eyes)**: These sympt

In [ ]:
!pip install fastapi uvicorn==0.22.0 pydantic python-multipart

In [ ]:
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

app = FastAPI(title="Measles Chatbot API")

# --- Global Variables for Model ---
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_path = "./mistral-7b-measles-qlora-adapter" # Path to your saved fine-tuned adapter
model = None
tokenizer = None

# --- Request Data Model ---
class ChatRequest(BaseModel):
    message: str
    history: list = [] # Optional: for multi-turn conversation context

# --- Startup Event: Load Model ---
@app.on_event("startup")
async def load_model():
    global model, tokenizer
    print("Loading model and tokenizer...")

    # 1. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)

    # 2. QLoRA Config (Same as training)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # 3. Load Base Model
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # 4. Load Fine-Tuned Adapter
    model = PeftModel.from_pretrained(base_model, adapter_path)
    print("Model loaded successfully!")

# --- Helper: Dummy RAG Retriever ---
def retrieve_idsr_context(query: str):
    """
    TODO: Replace this with your actual Vector DB search (FAISS/Chroma).
    For now, we return a static snippet from your guidelines.
    """
    # This simulates finding relevant sections in the IDSR docs
    return """
    IDSR Guideline Section 11:
    A suspected measles case is any person with fever and maculopapular (non-vesicular)
    generalized rash and cough, coryza or conjunctivitis (red eyes).
    """

# --- API Endpoint: /chat ---
@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    global model, tokenizer

    user_query = request.message

    # 1. Retrieve Context (RAG Step)
    context = retrieve_idsr_context(user_query)

    # 2. Format Prompt (MUST match fine-tuning format)
    prompt = f"""### Instruction:
{user_query}

### Context:
{context}

### Response:
"""

    # 3. Tokenize and Generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )

    # 4. Decode Response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the newly generated part (after "### Response:")
    if "### Response:" in full_response:
        final_answer = full_response.split("### Response:")[-1].strip()
    else:
        final_answer = full_response

    return {"response": final_answer, "context_used": context}

# --- Health Check ---
@app.get("/health")
async def health_check():
    return {"status": "active", "model_loaded": model is not None}

/tmp/ipython-input-2962948943.py:21: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [ ]:
# Install ngrok
!pip install pyngrok

In [ ]:
import torch
import uvicorn
import nest_asyncio
from threading import Thread
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pyngrok import ngrok

# --- 1. Define the FastAPI App ---
app = FastAPI(title="Measles Chatbot API")

class ChatRequest(BaseModel):
    message: str
    history: list = []

# This endpoint uses the 'model' and 'tokenizer' you loaded earlier in the notebook
@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    # Check if model is loaded in the notebook
    if 'model' not in globals() or 'tokenizer' not in globals():
        return {"response": "Error: Model not loaded in memory. Please run the fine-tuning/loading cells first."}

    user_query = request.message

    # Simple formatting (ensure this matches your training format)
    prompt = f"### Instruction:\n{user_query}\n\n### Response:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    if "### Response:" in full_response:
        final_answer = full_response.split("### Response:")[-1].strip()
    else:
        final_answer = full_response

    return {"response": final_answer}

@app.get("/health")
def health():
    return {"status": "ok"}

# --- 2. Start the Server & Tunnel ---
nest_asyncio.apply()

def run_server():
    # host="127.0.0.1" prevents the IPv6 error
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Stop any existing threads/tunnels to prevent conflicts
ngrok.kill()

# Start server in background
thread = Thread(target=run_server)
thread.start()

# Open the public tunnel
# Make sure your token is set! If not, uncomment the line below:
ngrok.set_auth_token("36eSFPyRmXAfKsTQ7WRyqWpsmDj_25rrefuGQpKA8PJjhAA6e")

public_url = ngrok.connect(8000).public_url
print(f"🚀 API is live at: {public_url}/chat")

INFO:     Started server process [3684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


🚀 API is live at: https://briley-prevocalic-carlo.ngrok-free.dev/chat


In [ ]:
import os
import torch
import uvicorn
import nest_asyncio
from threading import Thread
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- STEP 1: RELOAD THE MODEL ---
print("⏳ Checking for model in memory...")

# Only load if not already loaded to save time
if 'model' not in globals():
    print("⚙️ Loading Mistral 7B... (This takes about 1-2 minutes)")

    base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
    # This is the folder name we used in the fine-tuning step
    adapter_path = "./mistral-7b-measles-qlora-adapter"

    # 4-bit Quantization Config (Critical for Colab T4 GPU)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Load Tokenizer & Base Model
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto"
    )

    # Load Adapter (if you ran the fine-tuning step previously)
    if os.path.exists(adapter_path):
        print(f"✅ Found fine-tuned adapter at: {adapter_path}. Loading it...")
        model = PeftModel.from_pretrained(base_model, adapter_path)
    else:
        print("⚠️ Fine-tuned adapter NOT found. Loading Base Model only.")
        model = base_model

    print("✅ Model loaded into memory!")
else:
    print("✅ Model is already loaded.")


# --- STEP 2: DEFINE THE API ---
app = FastAPI(title="Measles Chatbot API")

class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    user_query = request.message

    # Format prompt (Instruction Tuned Format)
    prompt = f"### Instruction:\n{user_query}\n\n### Response:\n"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True
        )

    # Decode
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Parse out the answer
    if "### Response:" in full_text:
        answer = full_text.split("### Response:")[-1].strip()
    else:
        answer = full_text

    return {"response": answer}

# --- STEP 3: START SERVER & TUNNEL ---
nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Clean up old tunnels
ngrok.kill()

# Start server thread
thread = Thread(target=run_server)
thread.start()

# Open Tunnel
# Ensure your token is set. If you restarted the runtime, uncomment and paste it here:
ngrok.set_auth_token("36eSFPyRmXAfKsTQ7WRyqWpsmDj_25rrefuGQpKA8PJjhAA6e")

try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 API IS LIVE AT: {public_url}/chat")
    print("You can now send POST requests to this URL from Postman.")
except Exception as e:
    print("Ngrok Error:", e)
    print("Check if your auth token is set correctly.")

⏳ Checking for model in memory...
✅ Model is already loaded.


INFO:     Started server process [3684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



🚀 API IS LIVE AT: https://briley-prevocalic-carlo.ngrok-free.dev/chat
You can now send POST requests to this URL from Postman.


In [ ]:
# --- 0. INSTALL DEPENDENCIES (If not already installed) ---
!pip install -q -U langchain langchain-community faiss-cpu sentence-transformers pypdf python-docx langchain-text-splitters
!pip install -q -U fastapi uvicorn pyngrok nest_asyncio peft transformers bitsandbytes

import os
import torch
import uvicorn
import nest_asyncio
import shutil
from threading import Thread
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from typing import List

# LangChain & RAG Imports
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Model Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# ==========================================
# 1. SETUP & LOAD RAG VECTOR INDEX
# ==========================================
print("🚀 STARTING SETUP...")

# Define Paths (Aligned with your Drive structure)
# Ensure you have run: drive.mount('/content/drive')
rag_folder_path = "/content/drive/MyDrive/Measles_Chatbot_Project/RAG Files"
vector_db_path = "/content/drive/MyDrive/Measles_Chatbot_Project/faiss_index_measles"

# Initialize Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def build_or_load_vector_db():
    # If index already exists, load it
    if os.path.exists(vector_db_path):
        print(f"📂 Found existing Vector DB at {vector_db_path}. Loading...")
        try:
            return FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"⚠️ Error loading index: {e}. Rebuilding...")

    print("⚙️ Building new Vector DB from 'RAG Files'...")
    documents = []

    # Check if folder exists
    if not os.path.exists(rag_folder_path):
        os.makedirs(rag_folder_path, exist_ok=True)
        print(f"⚠️ Created folder {rag_folder_path}. Please upload your files here!")
        return None

    # Load Files
    for filename in os.listdir(rag_folder_path):
        file_path = os.path.join(rag_folder_path, filename)
        if filename.endswith(".pdf"):
            try:
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
                print(f"   - Loaded PDF: {filename}")
            except Exception as e:
                print(f"   ⚠️ Error loading PDF {filename}: {e}")
        elif filename.endswith(".docx"):
            try:
                loader = Docx2txtLoader(file_path)
                documents.extend(loader.load())
                print(f"   - Loaded DOCX: {filename}")
            except Exception as e:
                 print(f"   ⚠️ Error loading DOCX {filename}: {e}")

    if not documents:
        print("❌ No documents found. Index will be empty.")
        return None

    # Split Text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    print(f"📊 Created {len(chunks)} chunks.")

    # Create & Save Vector Store
    vector_db = FAISS.from_documents(chunks, embedding_model)
    vector_db.save_local(vector_db_path)
    print(f"💾 Vector DB saved to {vector_db_path}")
    return vector_db

# Initialize Vector DB
vector_store = build_or_load_vector_db()

# ==========================================
# 2. LOAD LLM (MISTRAL)
# ==========================================
print("⚙️ Loading Mistral 7B Model...")

# 1. Config
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# 2. Tokenizer & Base Model
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 3. Load Adapter (Update path if you have trained one)
# adapter_path = "/content/drive/MyDrive/Measles_Chatbot_Project/final_mistral_adapter"
# if os.path.exists(adapter_path):
#     print(f"✅ Loading Adapter from {adapter_path}")
#     model = PeftModel.from_pretrained(base_model, adapter_path)
# else:
print("ℹ️ Using Base Mistral Model (Adapter not found/configured)")
model = base_model

print("✅ Model Ready!")

# ==========================================
# 3. DEFINE API (FastAPI)
# ==========================================
app = FastAPI(title="ZNPHI Measles Chatbot")

class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    user_query = request.message

    # 1. Retrieve Context
    context_text = ""
    source_docs = []

    if vector_store:
        results = vector_store.similarity_search(user_query, k=3)
        for doc in results:
            context_text += f"\n- {doc.page_content}"
            source_docs.append(doc.metadata.get("source", "unknown"))
    else:
        context_text = "No context available."

    # 2. Format Prompt
    prompt = f"""### Instruction:
You are an expert medical assistant for measles surveillance in Zambia.
Use the Context below to answer the Question strictly based on the guidelines.
If the answer is not in the context, say "I cannot find that information in the guidelines."

### Context:{context_text}

### Question:
{user_query}

### Response:
"""

    # 3. Generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.3,
            do_sample=True
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Response:" in full_response:
        answer = full_response.split("### Response:")[-1].strip()
    else:
        answer = full_response

    return {
        "response": answer,
        "sources": list(set(source_docs))
    }

# ==========================================
# 4. START SERVER (Ngrok + Uvicorn)
# ==========================================
nest_asyncio.apply()

def run_server():
    # Force IPv4 to avoid "Connection Refused"
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Kill old tunnels
ngrok.kill()

# Start Server Thread
thread = Thread(target=run_server)
thread.start()

# Set Auth Token (Uncomment and add your token if needed)
ngrok.set_auth_token("36eSFPyRmXAfKsTQ7WRyqWpsmDj_25rrefuGQpKA8PJjhAA6e") # <--- REPLACE WITH YOUR NGROK AUTH TOKEN

try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n🚀 API IS LIVE AT: {public_url}/chat")
    print("Use Postman with method: POST")
except Exception as e:
    print("Ngrok Error:", e)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-16' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:63> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 161, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): [errno 98] address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-3806979170.py", 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.20.0 requires fastapi<0.119.0,>=0.115.0, but you have fastapi 0.124.2 which is incompatible.
🚀 STARTING SETUP...


/tmp/ipython-input-4033896288.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


📂 Found existing Vector DB at /content/drive/MyDrive/Measles_Chatbot_Project/faiss_index_measles. Loading...
⚙️ Loading Mistral 7B Model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ℹ️ Using Base Mistral Model (Adapter not found/configured)
✅ Model Ready!


INFO:     Started server process [3684]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.



🚀 API IS LIVE AT: https://briley-prevocalic-carlo.ngrok-free.dev/chat
Use Postman with method: POST


In [2]:
import nbformat
import os

# Get the current notebook's path (assuming it's in a Colab environment)
# This might require some manual adjustment if the notebook is not saved yet
# For simplicity, let's assume the notebook file is the one you are currently running
# In a typical Colab environment, this might be tricky to get automatically, so we'll use a placeholder and instruct.

# --- IMPORTANT: Replace 'YOUR_NOTEBOOK_NAME.ipynb' with the actual name of your notebook file ---
notebook_name = "ZNPHI Chatbot_cleaned.ipynb" # Change this to your notebook's name (e.g., 'Measles_Chatbot.ipynb')

# If you've saved the notebook to Drive, you'll need the full path
# For example: notebook_path = "/content/drive/MyDrive/Colab Notebooks/YOUR_NOTEBOOK_NAME.ipynb"
# Or if it's not saved to Drive, it might be in the /content/ folder during runtime

# Let's try to get the current notebook path from Colab's environment, if possible
# This is a common hack but might not always work perfectly
import ipykernel
import requests
from requests.exceptions import ConnectionError

try:
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-')[-1].split('.')[0]
    # Using the kernel_id, try to infer the notebook path
    # This is highly speculative and may not work consistently across Colab versions
    # A safer approach is to ask the user to manually provide the path.
    print("Attempting to find notebook path automatically...")
    # This path might not be directly accessible. It's usually a temporary file.
    # We'll stick to the manual instruction for robustness.
    notebook_path = '/content/drive/MyDrive/ZNPHI Chatbot_cleaned.ipynb' # Will rely on user to set this or load directly if they save the output
    print("Automatic path detection failed or is not reliable. Please manually set `notebook_path`.")
except (FileNotFoundError, ConnectionError, Exception) as e:
    print(f"Could not automatically determine notebook path: {e}")
    print("Please provide the full path to your notebook file manually. E.g.,")
    print("notebook_path = '/content/drive/MyDrive/ZNPHI Chatbot_cleaned.ipynb'")
    notebook_path = '/content/drive/MyDrive/ZNPHI Chatbot_cleaned.ipynb'

# --- Manual path setting (UNCOMMENT AND EDIT IF AUTO-DETECTION FAILS) ---
# notebook_path = "/content/drive/MyDrive/Colab Notebooks/My_Measles_Chatbot_Project.ipynb"
# Make sure this path points to the .ipynb file you want to clean

if notebook_path is None:
    print("❗ **ACTION REQUIRED:** Please edit this cell and manually set the `notebook_path` variable to the full path of your `.ipynb` file in your Google Drive or local storage. Then re-run this cell.")
    # For demonstration, we'll try to find a default Colab notebook if the user hasn't saved it yet
    # This is a heuristic and might not work for all users or setups.
    try:
        # Look for the last modified .ipynb file in /content/
        # This assumes the user is working on the most recently modified notebook.
        files_in_content = [f for f in os.listdir('/content') if f.endswith('.ipynb')]
        if files_in_content:
            files_in_content.sort(key=lambda x: os.path.getmtime(os.path.join('/content', x)), reverse=True)
            notebook_path = os.path.join('/content', files_in_content[0])
            print(f"⭐ **Heuristic:** Assuming your current notebook is: {notebook_path}")
            print("If this is not correct, please manually set `notebook_path` above.")
        else:
            print("Could not find any .ipynb files in /content/. Please ensure your notebook is saved.")
            raise ValueError("Notebook path not found.")
    except Exception as e:
        print(f"Error with heuristic path detection: {e}")
        print("Cannot proceed without a valid notebook path. Please set `notebook_path` manually.")
        # Exit or raise error if path cannot be determined
        raise RuntimeError("Cannot proceed without a valid notebook path.")


print(f"Loading notebook from: {notebook_path}")

with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

cleaned_nb = nb

# Iterate through cells and remove 'metadata.widgets' if present
for cell in cleaned_nb.cells:
    if 'metadata' in cell and 'widgets' in cell['metadata']:
        del cell['metadata']['widgets']
        print(f"Cleaned metadata.widgets from cell type: {cell.cell_type}")

# Save the cleaned notebook back to a new file (or overwrite, but new file is safer)
cleaned_notebook_path = notebook_path.replace('.ipynb', '_cleaned.ipynb')
with open(cleaned_notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(cleaned_nb, f)

print(f"✅ Notebook cleaned and saved as: {cleaned_notebook_path}")
print("You can now download this '_cleaned.ipynb' file and upload it to GitHub, or use Colab's 'Save a copy in GitHub' feature with this new file.")

Attempting to find notebook path automatically...
Automatic path detection failed or is not reliable. Please manually set `notebook_path`.
Loading notebook from: /content/drive/MyDrive/ZNPHI Chatbot_cleaned.ipynb
✅ Notebook cleaned and saved as: /content/drive/MyDrive/ZNPHI Chatbot_cleaned_cleaned.ipynb
You can now download this '_cleaned.ipynb' file and upload it to GitHub, or use Colab's 'Save a copy in GitHub' feature with this new file.
